# DeviaMetric - Calculateur de RMSD 

Bienvenue dans DeviaMetric ! Cet outil est conçu pour calculer le RMSD (Root Mean Square Deviation) entre deux fichiers PDB, tout en offrant une visualisation interactive de l'alignement entre les structures moléculaires correspondantes.

---

## À propos de DeviaMetric 🌟

Le RMSD est une mesure couramment utilisée en bio-informatique pour évaluer la similarité structurelle entre deux ensembles d'atomes. Plus précisément, il quantifie la moyenne des distances entre les atomes de référence (structure fixe) et les atomes mobiles (structure en cours d'évaluation), après une superposition optimale.

DeviaMetric propose une interface conviviale permettant de charger deux fichiers PDB, de choisir les atomes sur lesquels baser l'alignement (par exemple, tous les atomes, uniquement le squelette de la protéine, ou les atomes carbone-alpha), de visualiser l'alignement en 3D à l'aide de NGLView, et enfin de calculer le RMSD correspondant.

---

## Instructions d'utilisation 📝

1. **Chargement des Fichiers PDB :** Utilisez les cellules de code appropriées pour charger deux fichiers PDB.
2. **Spécification des Atomes pour l'Alignement :** Déterminez quels atomes utiliser pour l'alignement (par défaut, tous les atomes sont sélectionnés).
3. **Exécution :** Exécutez les cellules pour calculer le RMSD et visualiser l'alignement.

---

## Configuration Requise 🔧

Avant de commencer, assurez-vous d'avoir installé les dépendances suivantes :
- [Biopython](https://biopython.org/)
- [NGLView](https://github.com/nglviewer/nglview)

---

**Remarque :** Ce notebook est interactif et nécessite une exécution cellule par cellule pour afficher les résultats.


## I. Chargement des librairies

In [4]:
import nglview as nv
import numpy as np
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB import PDBParser, Superimposer
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output    
from IPython.display import display
import ipywidgets as widgets
import warnings


In [17]:
# pour ignorer les warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

## II. Chargement des structures

Par recherche dans la [RCSB PDB](https://www.rcsb.org/) :

In [ ]:
# Créer les widgets pour les deux ID PDB
pdb_id_1 = widgets.Text(
    value='',
    placeholder='Entrez le premier ID PDB',
    description='ID PDB 1:',
    disabled=False
)
display(pdb_id_1)

pdb_id_2 = widgets.Text(
    value='',
    placeholder='Entrez le deuxième ID PDB',
    description='ID PDB 2:',
    disabled=False
)
display(pdb_id_2)


# Fonction à appeler lors du clic sur le bouton
def visualisation_id_pdb(b):
    if isinstance(pdb_id_1, widgets.Text) and isinstance(pdb_id_2, widgets.Text):
            pdb_id1 = pdb_id_1.value.lower()
            print(pdb_id1, len(pdb_id1))
            pdb_id2 = pdb_id_2.value.lower()
            if pdb_id1 != '' and pdb_id2 != '':
                if len(pdb_id1) == 4 and len(pdb_id2) == 4 :
                    view1 = nv.show_pdbid(pdb_id1)
                    view2 = nv.show_pdbid(pdb_id2)
                    return view1, view2
                else:
                    print("Les ID PDB ne sont pas valides")
                    return
            else:
                print('Veuillez entrer les deux ID PDB')

# Créer un bouton pour valider les ID PDB
button = widgets.Button(description="Charger les structures PDB",
                        layout=widgets.Layout(width='200px'))
button.on_click(visualisation_id_pdb)
display(button)


Par chargement des fichiers :

In [ ]:
pdb_file_1 = widgets.FileUpload(
    description = 'Charger le 1er fichier PDB',
    accept='.pdb',
    layout=widgets.Layout(width='220px'),
    multiple=False  
)
display(pdb_file_1)

pdb_file_2 = widgets.FileUpload(
    description = 'Charger le 2nd fichier PDB',
    accept='.pdb',
    layout=widgets.Layout(width='220px'),
    multiple=False
)
display(pdb_file_2)

# Fonction à appeler lors du clic sur le bouton
def visualisation_fichier_pdb(b):
    pdb_1 = pdb_file_1.value
    pdb_2 = pdb_file_2.value
    print(pdb_1, pdb_2, len(pdb_1))
    if isinstance(pdb_1, widgets.FileUpload) and isinstance(pdb_2, widgets.FileUpload):
        if pdb_1 != None and pdb_2 != None:
            # Visualiser avec nv
            view1 = nv.show_file(pdb_1)
            view2 = nv.show_file(pdb_2)
            return view1, view2
    else:
        print('Veuillez charger les deux fichiers PDB')
    

display(button)
button.on_click(visualisation_fichier_pdb)


In [6]:
ficher1 = "1ake.pdb"
fichier2 = "4ake.pdb"

## III. Alignement des structures

In [12]:
# widget pour le choix des atomes de l'alignement
option_ali = widgets.ToggleButtons(
    options=['CA', 'backbone', 'all'],
    description='Alignement sur :',
    disabled=False,
    button_style='', 
    tooltips=['Alignement se fera sur les carbones alphas des 2 structures',
              'Alignement se fera sur les backbones des 2 structures',
              'Alignement se fera sur tous les atomes des 2 structures']
)
option_ali

ToggleButtons(description='Alignement sur :', options=('CA', 'backbone', 'all'), tooltips=('Alignement se fera…

In [13]:
print(option_ali.value)

CA


In [14]:
def align_pdb_files(file1, file2, align_type):
        parser = PDBParser()
        structure1 = parser.get_structure('structure1', file1)
        structure2 = parser.get_structure('structure2', file2)

        atoms1 = []
        atoms2 = []
        for model1, model2 in zip(structure1, structure2):
            for chain1, chain2 in zip(model1, model2):
                for residue1, residue2 in zip(chain1, chain2):
                    if align_type == "CA":
                        try:
                            atoms1.append(residue1['CA'])
                            atoms2.append(residue2['CA'])
                        except KeyError:
                            pass
                    elif align_type == "all":
                        atoms1.extend(residue1.get_atoms())
                        atoms2.extend(residue2.get_atoms())
                    elif align_type == "chain":
                        atoms1.extend(chain1.get_atoms())
                        atoms2.extend(chain2.get_atoms())

        sup = Superimposer()
        sup.set_atoms(atoms1, atoms2)
        sup.apply(structure2.get_atoms())

        return structure1, structure2

In [18]:
structure1, structure2 = align_pdb_files(ficher1, fichier2, option_ali.value)

/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3796.
  warnings.warn(
/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3860.
  warnings.warn(
/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3917.
  warnings.warn(
/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4158.
  warnings.warn(
/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3671.
  warnings.warn(
/home/tess01hp/.local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous

# IV. Calcul du RMSD

In [19]:
# Fonction pour calculer le RMSD entre deux structures alignées
def calculate_rmsd(structure1, structure2, align_type="CA"):
        atoms1 = []
        atoms2 = []

        for model1, model2 in zip(structure1, structure2):
            for chain1, chain2 in zip(model1, model2):
                for residue1, residue2 in zip(chain1, chain2):
                    if align_type == "CA":
                        try:
                            atoms1.append(residue1['CA'])
                            atoms2.append(residue2['CA'])
                        except KeyError:
                            pass
                    elif align_type == "all":
                        atoms1.extend(residue1.get_atoms())
                        atoms2.extend(residue2.get_atoms())
                    elif align_type == "chain":
                        atoms1.extend(chain1.get_atoms())
                        atoms2.extend(chain2.get_atoms())

        total_atoms = len(atoms1)
        rmsd = sum([(atom1 - atom2) ** 2 for atom1, atom2 in zip(atoms1, atoms2)]) / total_atoms
        rmsd = rmsd ** 0.5

        return rmsd

In [20]:
rmsd = calculate_rmsd(structure1, structure2)
print(f"Le RMSD entre la structure {structure1} et {structure2} est égale à {rmsd:.4f} Å")

Le RMSD entre la structure <Structure id=structure1> et <Structure id=structure2> est égale à 18.4487 Å
